There are a few issues with loading the data, some improperly formatted rows, if choosing to ignore the bad rows it contains 9373 rows in total from a CSV that had 10688 rows, rather large amount due to mismatch between number of items on a line and the number of columns, due to line breaks in the descriptions only 17 of the lines were badly formatted.

A cause for concern there is tweet data for 9662 users but the user table only contains 9380 users.

In [1]:
from gender_detector.gender_detector import GenderDetector as gd
import numpy as np
import pandas as pd
import re, json
import separateGUsers as sgu

twitter_Users = pd.read_csv("usersTwitter.csv", sep=",", encoding="utf-8", index_col=False, usecols=range(20))
#twitter_Users.head()
#twitter_Users
twitter_Users.shape

(9380, 20)

# Check For Duplicate Users and Tweets

In [38]:
twitter_Users.duplicated().value_counts()

False    8610
True      770
dtype: int64

770 duplicate rows in the user frame, will need to be dropped

Take list of google users, compare to twitter users to ensure one to one correlation and removing pages and keeping people. Select columns that are of interest and relevant to the model and write to file

__Don't run cell below more than once or if you do re-run the cells below this one__

In [15]:
users = sgu.read_dict_pairs()

twitter_users = pd.read_csv("usersTwitter.csv", sep=",", encoding="utf-8", index_col=False, usecols=range(20))
twFix = twitter_users[twitter_users.screen_name.isin(users.values())].drop_duplicates()
twFix = twFix[["id_str", "name", "screen_name", "location", "description", "url", "followers_count", "friends_count", "listed_count", "statuses_count", "lang"]]
twFix.set_index("id_str").to_csv("usersTrueTwitter.csv", index_label="id_str", encoding="utf-8")


Read in filtered twitter users

In [3]:
users = pd.read_csv("usersTrueTwitter.csv", sep=",", index_col="id_str", encoding="utf-8", engine='python')
users.shape

(7004, 7)

Testing gender guesser. US was selected because it is likely to include spanish names as well

In [17]:
test = gd("us")
test.guess("Emilia Miller")
name = users.iloc[0,0]
fname = name.split()[0]
test.guess(fname) #Justin
test.guess("the")

'male'

Try out regular expression match first token of only characters as some punctuation or numbers is mixed with it.

In [18]:
reg = re.compile("[a-z]*", re.IGNORECASE)
match = reg.match("Ber.t J. Gives, Jr.")


Assign genders to the twitter users using the package. Haven't created checks for accuracy but should. The name string is tokenised and the first token is match to the regexp and guessed. Sometimes the thing throws KeyErrors (disallowed chars) and IndexErrors (empty string).

__Takes a while to run so please save yourself the pain and load in the file I've already prepped. If you overwrite re-run this cell__

In [19]:
genderlist = []
gend = gd("us")
reg = re.compile("[a-z]*", re.IGNORECASE)
for name in users.name:
    try:
        if name is None:
            raise KeyError
        name = name.split()[0] #Take the first name prior to whitespace
        name = reg.match(name).group()
        gender = gend.guess(name)
        genderlist.append(gender)
    except (KeyError, IndexError):
        genderlist.append("unknown")
print("Done")

Done


Write the dataframe to a file, too time consuming to run all these cells whenever I want to start

In [20]:
users['gender'] = genderlist
users.to_csv("usersTrueTwitter.csv", index_label="id_str", encoding="utf-8")

Read in both dataframes for the user data __START HERE__

In [11]:
usersG = pd.read_csv("usersGoogle.csv", sep=",", index_col="id", quotechar="\"", encoding="utf-8", engine="c")
usersT = pd.read_csv("usersTrueTwitter.csv", sep=",", index_col="id_str", encoding="utf-8", engine='python')

Encode the Gender Column for both dataframes, dictionary initialised beforehand to see one-hot encoding

In [12]:
encDict = {"unknown":0,"none":0,"other":0,"male":1,"female":2}
usersT['gender_enc'] = usersT.gender.apply(lambda x: encDict[str(x).lower()])
usersG['gender_enc'] = usersG.gender.apply(lambda x: encDict[str(x).lower()])

In [13]:
usersT = usersT[[ "name", "screen_name", "location", "description", "friends_count", "gender", "gender_enc"]]
usersT

,name,screen_name,location,description,friends_count,gender,gender_enc
id_str,,,,,,,
235020718,Justin Esparza,JooseBoks,h.b. so.cali,You can tell from my speech that I come from t...,132,male,1
22450750,"Bert J. Gives, Jr.",bkbert23,"Bklyn, NYC","Notary Public, NSA.\n\nForeclosure Assitance,\...",2515,unknown,0
8011132,mike95,mike95,NaN,NaN,56,male,1
96217215,Elidier Moya,elidier_,Costa Rica,"Ingeniero en sistemas de información, con una ...",279,unknown,0
98688710,Ashutosh Jha,ajha1,"Toronto, Canada & Delhi, India",Student of #Entrepreneurship & #Life; Projects...,208,male,1
139496318,Ignacio Perez,rizos46,Madrid,1990,609,male,1
227128132,Bárbara Botello,Barbara_Botello,NaN,"Diputada federal del PRI por Guanajuato, en la...",1666,unknown,0
64238125,Emilia Miller,EmiliaMaria76,"Douglassville, PA",~Oncology Nurse~ Pitbull mom -Alvernia '17-,280,female,2
17189517,persuede,persuede,Brooklyn,pretty is good,517,unknown,0


In [14]:
test1 = "{'familyName': 'R.', 'givenName': 'Rebecca'}"
test2 = "{'familyName': 'O'Neill', 'givenName': 'Michael'}"

regexp = re.sub(r'{\'familyName\': \'(.*)\', \'givenName\': \'(.*)\'}',r'{"familyName": "\1", "givenName": "\2"}',test1)

Find the full name of the person from extracting the name dictionary and concatenating first and last names. Initially thought json loads would be sufficient but names with apostrophes cause issues when formating for json strings so a regular expression was used instead

In [15]:
def gather_name(name):
    name = re.sub(r'{\'familyName\': \'(.*)\', \'givenName\': \'(.*)\'}',r'{"familyName": "\1", "givenName": "\2"}',name)
    name = json.loads(name)
    return str(name["givenName"] + " " + name["familyName"])


usersG["fullname"] = usersG.name.apply(lambda x:gather_name(x))

Find location from locations dictionary similar problem as previously but instead the location listed as primary was chosen, a user could have multiple locations which werent primary so it was substituted for none instead. If multiple location which were listed as primary (which there should not be) the first match was chosen.

In [16]:
def gather_location(location):
    pat = re.compile("'value': '(.*)', 'prim", re.IGNORECASE)
    if location:
        location = re.search(pat, location)
        if location:
            return location.group(1)
    return None
        
usersG["primary_location"] = usersG.placesLived.apply(lambda x: gather_location(x))

Append occupation and tagline to match the Twitter description equivalent, people tend to also post their occupation there

In [17]:
occ = usersG.occupation
tag = usersG.tagline

desc = []

for job, line in zip(occ, tag):
    if job == "None":
        job = ""
    if line == "None":
        line = ""
    desc.append(str(line + " " + job).strip())
usersG['description'] = desc

In [18]:
usersG = usersG[['fullname', 'displayName', 'primary_location', 'description', 'circledByCount', 'gender', "gender_enc"]]
usersG

,fullname,displayName,primary_location,description,circledByCount,gender,gender_enc
id,,,,,,,
104288964437721273491,Johnny Oliveira,Johnny Oliveira,"São Paulo, Brasil.","Think, Believe and Make it Happen. Technology ...",0,male,1
100301662419920433743,Justin Esparza,Justin Esparza,HB,,0,male,1
115233306327496577248,"Bert Gives, Jr","Bert Gives, Jr",Valley Stream,License Notary & Certified Loan Signing Agent ...,224,male,1
114450247448428643447,Michael Olivero,Michael Olivero,None,Software,None,male,1
106057007019277820892,Elidier Moya,Elidier Moya,None,,73,male,1
115538752986453113378,Ashutosh Jha,Ashutosh Jha,"Toronto, Canada","Army of One, One of Army Creative & Social Ent...",222,male,1
108359110716028319267,ignacio perez alvarez,ignacio perez alvarez,None,cocinero,0,male,1
117585428399967312212,Bárbara Botello,Bárbara Botello,None,,79,female,2
112929377664095396761,Emilia Miller,Emilia Miller,None,,1,female,2


Write to file one last time

In [19]:
usersT.to_csv("usersTrueTwitter.csv", index_label="id_str", encoding="utf-8")
usersG.to_csv("usersGoogle.csv", index_label="id", encoding="utf-8")